In [7]:
import requests
import re
import os

from bs4 import BeautifulSoup
import pandas as pd

import prep_parse
import utility as util

In [2]:
### 1.台股權值100大排行榜
url = 'http://jow.win168.com.tw/z/zm/zmd/zmdb.djhtm'
html = requests.get(url)
html.encoding = 'big5'
sp = BeautifulSoup(html.text, 'html.parser')
stock_ids = []
stock_names = []
stock_ratios = []
for item in sp.select('.t3t1 a'):
    stock = item.text
    match = re.search(r'^\d{4}', stock)
    stock_id = match.group()
    stock_name = stock.replace(stock_id, '')
    stock_ratio = item.parent.next_sibling.next_sibling.next_sibling.next_sibling.text.replace('%', '')
    stock_ids.append(stock_id)
    stock_names.append(stock_name)
    stock_ratios.append(stock_ratio)
df_taiwan100 = pd.DataFrame({'stock_id': stock_ids, 'stock_name': stock_names, 'stock_ratio': stock_ratios})

In [3]:
### 2.MSCI台股指數成分股
url = 'http://jow.win168.com.tw/z/zm/zmd/zmdc.djhtm'
html = requests.get(url)
html.encoding = 'big5'
sp = BeautifulSoup(html.text, 'html.parser')
stock_ids = []
stock_names = []
for item in sp.select('.t3t1 a'):
    stock = item.text
    match = re.search(r'^\d{4}', stock)
    stock_id = match.group()
    stock_name = stock.replace(stock_id, '')
    stock_ids.append(stock_id)
    stock_names.append(stock_name)
df_msci = pd.DataFrame({'stock_id': stock_ids, 'stock_name': stock_names})

In [4]:
### 交集
df_leader = pd.merge(df_taiwan100, df_msci, on=['stock_id', 'stock_name'], how='inner')
print('1&2', df_leader.shape)

1&2 (83, 3)


In [5]:
def get_stock_info():
    # res = requests.get('http://dts.twse.com.tw/opendata/t187ap03_L.csv')
    # res.encoding='big5'

    # read file
    file_path = os.path.join('download', 'stock_info', 't187ap03_L.csv')
    if not os.path.exists(file_path):
        print('**ERROR:', file_path, 'does not exist')
        return None
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # delete unused rows
    df = df.astype(str)
    df = df[df['公司代號'].apply(lambda s: len(s) == 4)]
    # reserve used columns
    df = df[['公司代號','產業別']]
    df = df.rename(columns={'公司代號':'stock_id', '產業別':'industry'})
    
    return df 

In [8]:
### 3.非金融股
stock_info = get_stock_info()
df = pd.merge(df_leader, stock_info, on=['stock_id'], how='inner')
df_not_bank = df[df['industry'] != '金融保險業']
print('3', df_not_bank.shape)

3 (65, 4)


In [ ]:
### 4.低股價
# price_limit = 100
# daily_price = prep_parse.parse_daily_price_all(util.get_last_update_date())

# df = df_not_bank.set_index('stock_id')
# df['fit'] = False
# for stock_id in df.index:
#     if daily_price.loc[stock_id, 'close'] < price_limit:
#         df.loc[stock_id, 'fit'] = True

# df = df[df['fit']]
# df_under_pirce_limit = df.drop('fit', axis=1)
# print('4', df_under_pirce_limit.shape)

In [10]:
### 同產業類別放一起
df = df_not_bank.sort_values(by='industry')

### 存檔
df.to_csv('target_ori.csv', index=False, encoding='utf-8-sig')